In [34]:
#https://www.dummies.com/programming/python/plotting-a-sound-file-in-ipython/
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
from scipy.fftpack import fft
import numpy as np
from gurobipy import *
from IPython.display import Audio
import os
import glob
import untangle

import xml.etree.ElementTree as et
import pandas as pd

In [ ]:
#conda install pandas

# Functions

In [ ]:
# Function for transforming a signal with fourier
def fourier_trans(data, rate, begin_time, end_time): 
    # initialising variables
    tp_count = len(data)
    values = np.arange(int(tp_count/2))
    time_period = tp_count/rate
    frequencies = values/time_period
    
    #at what intervals time points are sampled
    sampling_interval = 1/rate;
    
    #time points
    time = np.arange(begin_time, end_time, sampling_interval);
    
    #Frequency domain representation
    fourier_transform = fft(data)/tp_count #normalized the amplitude
    fourier_transform = fourier_transform[range(int(len(data)/2))] #exclude sampling frequency   
    return frequencies, abs(fourier_transform)

# TODO: Funktion überarbeiten damit man es auch versteht!
def new_fft(data, rate):
    N = len(data)

    # sample spacing
    T = 1.0 / rate
    x = np.linspace(0.0, N*T, N)

    yf = fft(data)
    freq = np.linspace(0.0, 1.0/(2.0*T), N//2)
    ampl = 2.0/N * np.abs(yf[0:N//2])
    return freq, ampl

# Frequency domain representation
def plot_freq_domain(freq, fourier, name):
    plt.figure(figsize=(20,5))
    plt.plot(freq, fourier)
    plt.xlabel('Frequency in Hz')
    plt.ylabel('Amplitude')
    plt.xlim(0, 1400)
    plt.title(name)
    plt.show()

# TODO: Warnung herausgeben wenn bei der Teilung der Samplerate eine Fliesskommazahl entsteht
def easy_downsampling(data, rate, sampl_fac):
    ds_data = []
    for i in range(len(data)):
        if (i%sampl_fac == 0):
            ds_data.append(data[i])
    ds_rate = int(rate/sampl_fac)
    return ds_data, ds_rate

#ToDo: startwert von 0 nicht möglich überarbeiten!!
def snip_wav(data, rate, start_sec, end_sec):
    start_point = int((rate * start_sec)-1)
    end_point = int((rate * end_sec)-1)
    #print('start_point:', start_point)
    #print('end_point:', end_point)
    snip_data = data[start_point:end_point]
    #print(snip_data)  
    return snip_data, rate

def metric(target_v, approx_v):
    norm_factor = np.linalg.norm(approx_v)
    if (norm_factor != 0):
        norm_v = approx_v/norm_factor
        diff = target_v - norm_v
        diff_sum = np.sum(np.abs(diff))
        return diff_sum
    else:
        print('Der Vektor kann nicht normalisiert werden.')
        return approx_v 

##  Set audio snippets and sample compression

In [ ]:
start_sec = 1
end_sec = 2.5
sampl_fac = 4

# Dictonary

In [ ]:
## NEW
## Read WAV File
#
#audio_files = []
#path_wav = "audiofile"
#path_xml = "annotation"
#
## Init dict (combining same pitches into one)
#ref_note = {}
#note = 0
#
#for filename in sorted(glob.glob(os.path.join(path_xml, '*.xml'))):
#    
#    # Read data from xml
#    obj = untangle.parse(filename)
#    audiofile = obj.instrumentRecording.globalParameter.audioFileName.cdata
#    pitch = obj.instrumentRecording.transcription.event.pitch.cdata
#    #onsetSec = obj.instrumentRecording.transcription.event.onsetSec.cdata
#    #offsetSec = obj.instrumentRecording.transcription.event.offsetSec.cdata
#    fretNumber = obj.instrumentRecording.transcription.event.fretNumber.cdata
#    stringNumber = obj.instrumentRecording.transcription.event.stringNumber.cdata
#    
#    # Read wav
#    wav_file = path_wav + '/' + audiofile
#    rate, data = wav.read(wav_file)
#    
#    # Snip and downsampling audio-file
#    data, rate = snip_wav(data, rate, start_sec, end_sec)
#    data, rate = easy_downsampling(data, rate, sampl_fac)
#    
#    # Calc FFT
#    #freq, fourier = fourier_trans(data, rate, float(onsetSec), float(offsetSec))
#    #freq, fourier = fourier_trans(data, rate, 0, 2.5)
#    #freq, fourier = fourier_trans(data, rate, 0, end_sec-start_sec)
#    freq, fourier = new_fft(data, rate)
#    
#    # Update Dict
#    #ref_note[filename] = {
#    ref_note[note] = {
#        "pitch": int(pitch),
#        "fret": int(fretNumber),
#        "string": int(stringNumber),
#        "rate": int(rate),
#        "data": data,
#        #"onset": float(onsetSec), 
#        #"offset": float(offsetSec), 
#        "freq": freq,
#        "fourier": fourier,
#    }
#    note+=1

## Dataframe 

In [47]:
#xml_file = 'annotation/G53-40100-1111-00001.xml'
xml_file = 'recordings/annotation/instrumental_lead.xml'
tree = et.parse(xml_file)
root = tree.getroot()

In [57]:
df_cols = ['audioFileName', 'pitch', 'onsetSec', 'fretNumber']
rows = []

for globalParam in root.findall('globalParameter'):
    file_name = globalParam.find('audioFileName').text

for transcription in root.findall('transcription'):
    #print('tran:',transcription)
    
    for event in transcription.findall('event'):
        #print('event:', event)
        res = []
        res.append(file_name)
        
        for elem in df_cols[1:]:
            if event is not None and event.find(elem) is not None:
                res.append(event.find(elem).text)
            else:
                res.append(None)
        
        rows.append({df_cols[i]: res[i] for i, _ in enumerate(df_cols)})
        #print('rows', rows)

    out_df = pd.DataFrame(rows, columns=df_cols)
out_df

,audioFileName,pitch,onsetSec,fretNumber
0,\instrumental_lead.wav,64,0.519,14
1,\instrumental_lead.wav,66,0.78,16
2,\instrumental_lead.wav,67,0.968,17
3,\instrumental_lead.wav,64,1.179,14
4,\instrumental_lead.wav,71,1.385,16
...,...,...,...,...
86,\instrumental_lead.wav,81,48.039,17
87,\instrumental_lead.wav,81,49.269,17
88,\instrumental_lead.wav,83,49.705,19
89,\instrumental_lead.wav,84,50.107,20


# Experiment with df and list

In [ ]:
# Read WAV File

audio_files = []
path_wav = "audiofile"
path_xml = "annotation"

# Init dict
ref_note = {}
note = 0
l_r_n = []

for filename in sorted(glob.glob(os.path.join(path_xml, '*.xml'))):
    
    # Read data from xml
    obj = untangle.parse(filename)
    audiofile = obj.instrumentRecording.globalParameter.audioFileName.cdata
    pitch = obj.instrumentRecording.transcription.event.pitch.cdata
    #onsetSec = obj.instrumentRecording.transcription.event.onsetSec.cdata
    #offsetSec = obj.instrumentRecording.transcription.event.offsetSec.cdata
    fretNumber = obj.instrumentRecording.transcription.event.fretNumber.cdata
    stringNumber = obj.instrumentRecording.transcription.event.stringNumber.cdata
    
    # Read wav
    wav_file = path_wav + '/' + audiofile
    rate, data = wav.read(wav_file)
    
    # Snip and downsampling audio-file
    data, rate = snip_wav(data, rate, start_sec, end_sec)
    data, rate = easy_downsampling(data, rate, sampl_fac)
    
    # Calc FFT
    #freq, fourier = fourier_trans(data, rate, float(onsetSec), float(offsetSec))
    #freq, fourier = fourier_trans(data, rate, 0, 2.5)
    #freq, fourier = fourier_trans(data, rate, 0, end_sec-start_sec)
    freq, fourier = new_fft(data, rate)
    
    # Update Dict
    #ref_note[filename] = {
    ref_note = {
        "pitch": int(pitch),
        "fret": int(fretNumber),
        "string": int(stringNumber),
        "rate": int(rate),
        "data": data,
        #"onset": float(onsetSec), 
        #"offset": float(offsetSec), 
        "freq": freq,
        "fourier": fourier,
    }
    l_r_n.append(ref_note)
    #note+=1

In [ ]:
#list_note = [ref_note]
#list_note
l_r_n

In [ ]:
import pandas as pd
df_ = pd.DataFrame(l_r_n)
df_

In [ ]:
columns = ['pitch', 'freq', 'fourier']
df__ = df_[columns]
df__.loc[df__['pitch'] == 45]


In [ ]:
#df__.groupby(['pitch']).agg(
#mean_freq = pd.NamedAgg(column='freq', aggfunc=sum),
#mean_fourier = pd.NamedAgg(column='fourier', aggfunc=sum))
grouped_df = df__.groupby(['pitch']).sum().reset_index()
grouped_df.loc[grouped_df['pitch'] == 45]


In [ ]:
index_cols = df_.columns.tolist()
print(index_cols)
index_cols.remove('pitch')
print(index_cols)

df__ = df_.groupby(index_cols)
df__

In [ ]:
# sum fourier-values 
combi = [(a+b)/2 for a, b in zip(note_dict[5]['fourier'], note_dict[6]['fourier'])]
print(combi)
len(combi)

In [ ]:
tone = 5
plot_freq_domain(note_dict[tone]['freq'], combi,'Pitch: %g' %ref_note[tone]['pitch'])
print('data:', len(ref_note[tone]['data']))
print('rate:', ref_note[tone]['rate'])

In [ ]:
tone = 5
plot_freq_domain(ref_note[tone]['freq'], ref_note[tone]['fourier'],'Pitch: %g' %ref_note[tone]['pitch'])
print('data:', len(ref_note[tone]['data']))
print('rate:', ref_note[tone]['rate'])

In [ ]:
tone = 6
plot_freq_domain(ref_note[tone]['freq'], ref_note[tone]['fourier'], 'Pitch: %g' %ref_note[tone]['pitch'])
print('data:', len(ref_note[tone]['data']))
print('rate:', ref_note[tone]['rate'])

# Read audiofile for approximation

In [ ]:
# Chords
#rate_chord, data_chord = wav.read('audiofile/1-E1-Major 00.wav')
#rate_chord, data_chord = wav.read('audiofile/1-E1-Major 01.wav')

# Single note
#rate_chord, data_chord = wav.read('audiofile/G53-65601-1111-00067.wav')
rate_chord, data_chord = wav.read('audiofile/G53-40100-1111-00001.wav')

# Single note played on another guitar
#rate_chord, data_chord = wav.read('audiofile/G53-42102-1111-237.wav')

In [ ]:
# mischsignal mischsignal
data_20, rate_20 = snip_wav(data_chord, rate_chord, start_sec, end_sec)
data_ds, rate_ds = easy_downsampling(data_20, rate_20, sampl_fac)
freq, mischsignal = new_fft(data_ds, rate_ds)
#freq, mischsignal = new_fft(data_chord, rate_chord)

plot_freq_domain(freq, mischsignal, 'Mischsignal')
#print('data:', len(data_ds))
#print('rate:', rate_ds)
print('Länge Mischsignal: ', len(mischsignal))

# Gurobi

In [ ]:
# Build Model
m = Model('AMt')

lambs = []
for lamb in range(len(ref_note)):
    lambs.append(m.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'lamb_' + str(lamb)))
m.update()

In [ ]:
'''
%%time
#Optimize Signal

deviation = 0
approxi_sgn = 0
 
for sgn in range(len(mischsignal)):
#for sgn in range(100):
    for lamb in range(len(lambs)):
        approxi_sgn += lambs[lamb] * ref_note[lamb]['fourier'][sgn]
        
    deviation += ((mischsignal[sgn] - approxi_sgn)*(mischsignal[sgn] - approxi_sgn))
    # deviation wird ausmultipliziert stattdessen die für 'mischsignal[sgn] - approxi_sgn'
    # eine Gurobi-Variable (m.addVar  inkl. constr.) verwenden
    # Überprüfen ob der Betrag der Differenz besser geeignet ist?
print('Deviation is calculated.')
'''

In [ ]:
%%time
# Schlaufen vertauscht
deviation = 0
approxi_sgn = 0

for lamb in range(len(lambs)):
    la = lambs[lamb]
    print('la:', la)
    note = ref_note[lamb]['fourier']
    
    for sgn in range(len(mischsignal)):
        approxi_sgn = la * note[sgn]
        mi = mischsignal[sgn]
        deviation += ((mi - approxi_sgn)*(mi - approxi_sgn))
print('Deviation is calculated.')

In [ ]:
%%time
m.setObjective(deviation, GRB.MINIMIZE)
m.optimize()

approx_v = []

for v in m.getVars():
    print('%s: %g' % (v.varName, v.x))
    approx_v.append(v.x)

# Metrik berechnen

In [ ]:
target_v = np.zeros(78,dtype=int)
target_v[0] = 1

In [ ]:
metric(target_v, approx_v)

# Test Area

In [35]:
path_xml = "recordings/annotation"
 
audio_input = {}
 
for filename in glob.glob(os.path.join(path_xml, '*.xml')):
    obj = untangle.parse(filename)
    audiofile = obj.instrumentRecording.globalParameter.audioFileName.cdata
    event = obj.instrumentRecording.transcription.event
   
    i=0
    event_dict = {}
   
    for e in event:
        pitch = e.pitch.cdata
        onsetSec = e.onsetSec.cdata
        event_dict['event_' + str(i)] = {
            "pitch": int(pitch),
            "onset": float(onsetSec)
        }
        i+=1
   
    audio_input[audiofile] = event_dict
audio_input

{'\\instrumental_lead.wav': {'event_0': {'pitch': 64, 'onset': 0.519},
  'event_1': {'pitch': 66, 'onset': 0.78},
  'event_2': {'pitch': 67, 'onset': 0.968},
  'event_3': {'pitch': 64, 'onset': 1.179},
  'event_4': {'pitch': 71, 'onset': 1.385},
  'event_5': {'pitch': 71, 'onset': 2.635},
  'event_6': {'pitch': 72, 'onset': 3.042},
  'event_7': {'pitch': 69, 'onset': 3.263},
  'event_8': {'pitch': 71, 'onset': 3.491},
  'event_9': {'pitch': 72, 'onset': 3.687},
  'event_10': {'pitch': 75, 'onset': 3.907},
  'event_11': {'pitch': 76, 'onset': 5.346},
  'event_12': {'pitch': 84, 'onset': 7.233},
  'event_13': {'pitch': 83, 'onset': 7.873},
  'event_14': {'pitch': 81, 'onset': 8.078},
  'event_15': {'pitch': 79, 'onset': 8.656},
  'event_16': {'pitch': 78, 'onset': 8.86},
  'event_17': {'pitch': 79, 'onset': 9.062},
  'event_18': {'pitch': 78, 'onset': 9.254},
  'event_19': {'pitch': 76, 'onset': 10.137},
  'event_20': {'pitch': 75, 'onset': 10.548},
  'event_21': {'pitch': 76, 'onset': 1

In [41]:
df = pd.DataFrame.from_dict(audio_input, orient='index').T
df

,\instrumental_lead.wav,instrumental_rhythm2.xml,\mim-riff1-short-slow.wav
event_0,"{'pitch': 64, 'onset': 0.519}","{'pitch': 40, 'onset': 0.35}","{'pitch': 51, 'onset': 0.293}"
event_1,"{'pitch': 66, 'onset': 0.78}","{'pitch': 52, 'onset': 0.35}","{'pitch': 53, 'onset': 0.617}"
event_2,"{'pitch': 67, 'onset': 0.968}","{'pitch': 59, 'onset': 0.35}","{'pitch': 51, 'onset': 0.96}"
event_3,"{'pitch': 64, 'onset': 1.179}","{'pitch': 40, 'onset': 0.79}","{'pitch': 56, 'onset': 1.254}"
event_4,"{'pitch': 71, 'onset': 1.385}","{'pitch': 52, 'onset': 0.79}","{'pitch': 54, 'onset': 1.603}"
...,...,...,...
event_412,NaN,"{'pitch': 59, 'onset': 39.92}",NaN
event_413,NaN,"{'pitch': 64, 'onset': 39.92}",NaN
event_414,NaN,"{'pitch': 52, 'onset': 40.14}",NaN
event_415,NaN,"{'pitch': 59, 'onset': 40.14}",NaN
